In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Import your CRUD module
from animal_shelter import AnimalShelter

# Initialize the animal shelter CRUD object
# MongoDB connection credentials
username = "aacuser"
password = "SecurePassword123"
host = "nv-desktop-services.apporto.com"
port = 30325
db = "AAC"
collection = "animals"

# Connect to database via CRUD Module
db = AnimalShelter(username, password, host, port, db, collection)

# Class read method must support return of list object and accept projection json input
# Sending the read method an empty document requests all documents be returned
try:
    results = db.read({})

    # Create DataFrame from results - handle empty results
    if results:
        df = pd.DataFrame.from_records(results)
        
        # MongoDB v5+ is going to return the '_id' column with an invalid object type of 'ObjectID'
        # Remove it in the dataframe here to prevent crashes
        if '_id' in df.columns:
            df.drop(columns=['_id'], inplace=True)
    else:
        
        # Create empty DataFrame with expected columns if no results
        print("No results returned from database")
        df = pd.DataFrame(columns=['animal_type', 'breed', 'name', 'sex_upon_outcome', 
                                'age_upon_outcome_in_weeks', 'outcome_type',
                                'location_lat', 'location_long'])
except Exception as e:
    print(f"Error connecting to database: {e}")
    
    # Create empty DataFrame with expected columns if connection fails
    df = pd.DataFrame(columns=['animal_type', 'breed', 'name', 'sex_upon_outcome', 
                            'age_upon_outcome_in_weeks', 'outcome_type',
                            'location_lat', 'location_long'])

# Create the JupyterDash app
app = JupyterDash(__name__)

# Load the Grazioso Salvare logo
image_filename = 'Grazioso_Salvare_Logo.png' 
try:
    if os.path.exists(image_filename):
        encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode('ascii')
    else:
        print(f"Warning: Logo file '{image_filename}' not found")
        
        # Create a blank image as a fallback
        encoded_image = ""
except Exception as e:
    print(f"Error loading logo: {e}")
    encoded_image = ""

# Create app layout
app.layout = html.Div([
    
    # First row: Logo and header
    html.Div([
        html.Div([
            
            # Load the logo image or use a blank space if the file doesn't exist
            html.Img(src='data:image/png;base64,{}'.format(encoded_image), 
                     style={'height': '200px', 'width': '200px'} if encoded_image else {'display': 'none'}),
        ], style={'display': 'inline-block', 'width': '20%'}),
        html.Div([
            html.Center(html.H1('Grazioso Salvare - Rescue Dog Finder')),
            html.Center(html.H3('Dashboard created by Tristen Bradney - April 2025'))
        ], style={'display': 'inline-block', 'width': '60%'})
    ]),
    
    html.Hr(),
    
    # Second row: Radio buttons for filtering
    html.Div([
        html.Center(html.H3('Filter by Rescue Type:')),
        html.Center(dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset Filter', 'value': 'reset'}
            ],
            value='reset',
            labelStyle={'display': 'inline-block', 'margin-right': '20px'}
        ))
    ]),
    
    html.Hr(),
    
    # Third row: Data table
    html.Div([
        dash_table.DataTable(
            id='datatable-id',
            columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
            data=df.to_dict('records'),
            
            # Number of rows per page
            page_size=10,  
            style_data={
                'whiteSpace': 'normal',
                'height': 'auto',
                'lineHeight': '15px'
            },
            style_cell={'textAlign': 'left', 'padding': '5px'},
            style_header={
                'backgroundColor': 'rgb(30, 30, 30)',
                'color': 'white',
                'fontWeight': 'bold'
            },
            
            # Enable sorting
            sort_action="native", 
            
            # Enable multi-column sorting
            sort_mode="multi",  
            
            # Enable filtering
            filter_action="native", 
            
            # Allow row selection
            row_selectable="single", 
            
            # Initially select first row
            selected_rows=[0]  
        )
    ]),
    
    html.Br(),
    html.Hr(),
    
    # Fourth row: Charts side by side
    html.Div(
        className='row',
        style={'display': 'flex'},
        children=[
            
            # Breed distribution chart
            html.Div(
                id='graph-id',
                className='col s12 m6',
            ),
            
            # Geolocation map
            html.Div(
                id='map-id',
                className='col s12 m6',
            )
        ]
    )
])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback for updating the data table based on filter selection
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    
    # Define the query based on the rescue type
    if filter_type == 'water':
        
        # Water Rescue criteria
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'mountain':
        
        # Mountain or Wilderness Rescue criteria
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'disaster':
        
        # Disaster or Individual Tracking criteria
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
    else:
        
        # Reset - show all dogs
        query = {"animal_type": "Dog"}
    
    # Use the read method from our CRUD module to get the data
    dogs = db.read(query)
    
    # Handle the case of no results
    if not dogs:
        return []
    
    # Convert to pandas DataFrame and handle _id column if present
    dogs_df = pd.DataFrame.from_records(dogs)
    if '_id' in dogs_df.columns:
        dogs_df.drop(columns=['_id'], inplace=True)
    
    # Return the dogs that match the query
    return dogs_df.to_dict('records')

# This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Display the breeds of animal based on quantity represented in the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    
    # If no data, return empty
    if not viewData:
        return []
    
    # Convert to dataframe
    dff = pd.DataFrame(viewData)
    
    # Count breeds
    if 'breed' in dff.columns and not dff.empty:
        breed_counts = dff['breed'].value_counts().reset_index()
        breed_counts.columns = ['breed', 'count']
        
        # If there are too many breeds, group the less common ones as "Other"
        if len(breed_counts) > 10:
            top_breeds = breed_counts.iloc[:10]
            other_count = breed_counts.iloc[10:]['count'].sum()
            top_breeds = pd.concat([top_breeds, pd.DataFrame({'breed': ['Other'], 'count': [other_count]})], ignore_index=True)
            breed_counts = top_breeds
        
        # Create pie chart
        fig = px.pie(
            breed_counts,
            values='count',
            names='breed',
            title='Dog Breed Distribution'
        )
        
        # Return the chart
        return [
            dcc.Graph(figure=fig)
        ]
    else:
        
        # Return empty if no breed column or empty dataframe
        return []

# This callback will update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index): 
    
    # Handle empty data cases
    if not viewData:
        return []
    
    # Handle no selection case
    if not index or len(index) == 0:
        return []
    
    try:
        dff = pd.DataFrame.from_dict(viewData)
        
        # Handle empty dataframe
        if dff.empty:
            return []
        
        # Get the selected row
        row = index[0]
        
        # Check if the location columns exist
        if ('location_lat' in dff.columns and 'location_long' in dff.columns and
            dff['location_lat'].iloc[row] and dff['location_long'].iloc[row]):
            
            # Create the map with the marker
            return [
                dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
                    dl.TileLayer(id="base-layer-id"),
                    
                    # Marker with tool tip and popup
                    dl.Marker(position=[dff.iloc[row, dff.columns.get_loc('location_lat')], 
                                       dff.iloc[row, dff.columns.get_loc('location_long')]], 
                             children=[
                                dl.Tooltip(dff.iloc[row, dff.columns.get_loc('breed')] if 'breed' in dff.columns else ""),
                                dl.Popup([
                                    html.H1("Animal Name"),
                                    html.P(dff.iloc[row, dff.columns.get_loc('name')] if 'name' in dff.columns else "Unknown")
                                ])
                             ])
                ])
            ]
        else:
            
            # Return a default map with no markers if location data is missing
            return [
                dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
                    dl.TileLayer(id="base-layer-id")
                ])
            ]
    except Exception as e:
        print(f"Error in map update: {e}")
        
        # Return a default map in case of any errors
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id")
            ])
        ]

# Run the app
app.run_server(debug=True)


Dash app running on http://127.0.0.1:28885/
